In [ ]:
import math
import time
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import sys

# Programmieraufgabe: CG-Verfahren und PCG-Verfahren

Betrachtet wird das lineare Gleichungssystem $Ax = b$, wobei
$$
 A =  \begin{bmatrix}
        \phantom{-}2 & -1 &  &  &  \\
        -1& \phantom{-}2 & -1&  & \\
        & \ddots & \ddots & \ddots & \\
        & & -1 & \phantom{-}2 & -1 \\
        & & & -1 & \phantom{-}2 
    \end{bmatrix} \in \mathbb{R}^{n\times n}.
$$
verwendet wird. Ziel ist es, dieses System mithilfe des **CG-Verfahrens** (konjugierte Gradienten) und des **PCG-Verfahrens** (vorkonditionierte Gradienten) zu lösen.

## Teil 1: Implementierung der Hilfsfunktionen

Zunächst sind die folgenden Funktionen zu implementieren, die eine explizite Speicherung der vollständigen Matrizen vermeiden:

* `Av = Av_mul(v)`: Diese Funktion soll die Multiplikation der Matrix $A$ mit einem beliebigen Vektor $v$ effizient realisieren, **ohne** dass $A$ als $n \times n$-Matrix gespeichert werden muss. (Wird auch als "Matrix-Free" bezeichnet.)
* `w = pre_mul(r)`: Analog soll diese Funktion die Multiplikation eines Vektors $r$ mit dem Vorkonditionierer $M^{-1} = S^{-1} S^{-T}$ realisieren.

Für die Vorkonditionierung soll, wie in der Vorlesung besprochen, $S = \frac{1}{2} D + L^T$ verwendet werden, wobei die Zerlegung $A = L + D + L^T$ zugrunde liegt ($L$ ist der strikte untere Dreiecksteil und $D$ die Diagonale von $A$). Dadurch hat $S^{-T}S^{-1}$ die Form
$$
\begin{bmatrix}
n & \dots & 3 & 2 & 1 \\
\vdots & & \vdots & \vdots & \vdots \\
3 & \dots & 3 & 2 & 1 \\
2 & \dots & 2 & 2 & 1 \\
1 & \dots & 1 & 1 & 1
\end{bmatrix}
$$

In [ ]:
def Av_mul(v):
    """
    A*v = Av_mul ( v )
    (Hier darf nicht die Matrix A angelegt werden.)
    """
    Av = 2.0 * v
    Av[:-1] -= v[1:]
    Av[1:] -= v[:-1]
    return Av

def pre_mul(r):
    """
    Anwendung des Vorkonditionierers S^(-T)S^(-1)
    (Auch hier dürfen weder A noch S^(-1) als Matrix angelegt werden)
    """
    n = r.shape[0]
    z = np.zeros(n)
    total = 0.0
    for j in range(0, n):
        total = total + r[j]
        z[j] = total
    w = np.zeros(n)
    total = 0.0
    for i in range(n - 1, -1, -1):
        total = total + z[i]
        w[i] = total
    return w


### Teil 2: Implementierung der CG-Verfahren

Als Nächstes werden die Hauptfunktionen für die iterativen Lösungsverfahren benötigt:

* `x, res = cg(b, avm, tol, kmax)`
* `x, res = pcg(b, avm, prm, tol, kmax)`

Diesen Funktionen werden `Av_mul` und `pre_mul` als Parameter (`avm` und `prm`) übergeben, um die Matrixoperationen durchzuführen.

Parameter:
* `tol` ist die Fehlertoleranz. Die Verfahren sollen abgebrochen werden, wenn **sowohl** die Norm des Residuums **als auch** die Norm der Differenz aufeinanderfolgender Iterierter $\leq$ `tol` ist.
* `kmax` ist die maximale Anzahl an Iterationen. Die Verfahren müssen spätestens terminieren, wenn diese Grenze erreicht ist.
* Als Startvektor wählen Sie die rechte Seite $b$.


Die Rückgabewerte der Funktionen `cg` und `pcg` sollen sein:
* `x`: Die vom jeweiligen Verfahren berechnete Lösung.
* `res` (Vektor): Der Vektor mit den Normen des Residuums:
    $$\text{res}(i) = \| b - A x^{(i)} \|_{2}$$


In [ ]:
def cg(b, avm, tol, max_iter):
    """
    Konjugierte-Gradienten-Verfahren (CG)
        solution, residuals, steps = cg(b, avm, tol, max_iter)

    Parameter:
      b        : Rechte Seite und Initialschätzung (Startvektor x0);
      avm      : Funktion, die A*v berechnet;
      tol      : Geforderte absolute Genauigkeit;
      max_iter : Maximale Zahl von Iterationen.

    Resultate:
      solution  : Näherungslösungsvektor xk von A*xk=b;
      residuals : Folge der Residuennormen ||b - A*x^(i)||_2;
      steps     : Folge der Schrittweitennormen ||x^(i) - x^(i-1)||_2.
      residuals und steps haben gleiche Länge. steps[0] ist np.nan.

    Das Verfahren bricht ab, wenn entweder die maximale Zahl von
    Iterationsschritten 'kmax' erreicht ist oder die 2-Normen von
    Residuum (A*x - b) und Schrittweite (x^(i+1) - x^(i)) beide kleiner-gleich
    der Toleranz 'tol' sind.
    """
    n = b.shape[0]

    residuals = np.zeros(max_iter + 1)  # Residuum  || A*x^(i) - b ||_2
    
    # Initialisierung
    iter_count = 0                           # Zählt die berechneten Iterationsschritte
    solution = np.copy(b)                    # x_0 = b
    residual = b - avm(solution)             # r_0 = b - A*x_0
    search_direction = np.copy(residual)     # p_0 = r_0
    gamma_old = np.inner(residual, residual) # gamma_0 = <r_0, r_0>

    residuals[iter_count] = math.sqrt(gamma_old) 

    while (iter_count < max_iter) and (
        (residuals[iter_count] > tol) or (step_norm > tol)
    ):
        # 1. Berechne A*p_k
        v = avm(search_direction)
        
        # 2. Berechne alpha_k (Schrittweite)
        omega = np.inner(search_direction, v) # <p_k, A*p_k>
        
        if abs(omega) < 1e-14:
            print(
                f"CG-Warnung: k={iter_count}/{max_iter} , gamma_old={format(gamma_old,'.2e')} , <p_k,v_k>={format(omega,'.2e')} < 1e-14 !!"
            )
            break

        alpha = gamma_old / omega # alpha_k = gamma_k / omega_k

        # 3. Update x_k+1
        step_vector = alpha * search_direction
        step_norm = norm(step_vector)
        solution += step_vector

        # 4. Update r_k+1
        residual -= alpha * v

        # 5. Berechne gamma_k+1
        gamma_new = np.inner(residual, residual) # gamma_k+1 = <r_k+1, r_k+1>

        # 6. Update p_k+1
        beta = gamma_new / gamma_old # beta_k = gamma_k+1 / gamma_k
        
        search_direction *= beta
        search_direction += residual
        
        # 7. Speichern und Update
        iter_count += 1
        
        # Residuen-Norm für den Abbruch: ||r_k+1||_2
        residuals[iter_count] = norm(residual)
        
        gamma_old = gamma_new

    # Die resultierenden Vektoren sollen genau die richtige Laenge haben;
    # alle ueberschuessigen Elemente werden abgeschnitten:
    residuals = residuals[: iter_count + 1]

    return (solution, residuals)

In [ ]:
def pcg(b, avm, prm, tol, max_iter):
    """
    Präkonditioniertes Konjugierte-Gradienten-Verfahren (PCG)
        solution, residuals, steps = pcg(b, avm, prm, tol, max_iter)

    Parameter:
      b : Rechte Seite und Initialschätzung (Startvektor x0);
      avm : Funktion, die A*v berechnet;
      prm : Funktion, die M^(-1)*v berechnet (M^(-1) = S^(-T)*S^(-1));
      tol : Geforderte absolute Genauigkeit;
      max_iter: Maximale Zahl von Iterationen.

    Resultate:
      solution   : Näherungslösungsvektor xk von A*xk=b;
      residuals : Folge der Residuennormen ||b - A*x^(i)||_2;
      steps     : Folge der Schrittweitennormen ||x^(i) - x^(i-1)||_2.
      residuals und steps haben gleiche Länge. steps[0] ist np.nan.
    """
    n = b.shape[0]

    # Initialisierung der Ergebnis-Vektoren (mit Platz für den Startwert, Index 0)
    residuals = np.zeros(max_iter + 1)  # Residuum  || A*x^(i) - b ||_2

    # Initialisierung
    iter_count = 0                     # Zählt die berechneten Iterationsschritte
    solution = np.copy(b)              # x_0 = b
    residual = b - avm(solution)       # r_0 = b - A*x_0
    
    # Präkonditionierung: w_0 = M^(-1) * r_0
    preconditioned_residual = prm(residual) # w_0 = M^(-1) * r_0
    
    # Suchrichtung: p_0 = w_0
    search_direction = np.copy(preconditioned_residual) 
    
    # Skalarprodukt: gamma_0 = <w_0, r_0>
    gamma_old = np.dot(preconditioned_residual, residual) 

    # Speichern der Werte für Iteration k=0
    residuals[iter_count] = norm(residual) 

    # Iteration
    while (iter_count < max_iter) and (
        (residuals[iter_count] > tol) or (step_norm > tol)
    ):
        # 1. Berechne A*p_k
        vec_v = avm(search_direction)
        
        # 2. Berechne alpha_k (Schrittweite)
        # omega = <p_k, A*p_k>
        omega = np.dot(search_direction, vec_v) 
        
        if abs(omega) < 1e-14:
            print(
                f"PCG-Warnung: k={iter_count}/{max_iter} , gamma_old={format(gamma_old,'.2e')} , <p_k,v_k>={format(omega,'.2e')} < 1e-14!"
            )
            break

        alpha = gamma_old / omega # alpha_k = gamma_k / omega_k

        # 3. Update x_k+1
        step_vector = alpha * search_direction
        step_norm = norm(step_vector)
        solution += step_vector

        # 4. Update r_k+1
        residual -= alpha * vec_v
        
        # 5. Präkonditionierung: w_k+1 = M^(-1) * r_k+1
        preconditioned_residual = prm(residual)
        
        # 6. Berechne gamma_k+1
        gamma_new = np.dot(preconditioned_residual, residual) # gamma_k+1 = <w_k+1, r_k+1>

        # 7. Update p_k+1
        beta = gamma_new / gamma_old # beta_k = gamma_k+1 / gamma_k
        
        search_direction *= beta
        search_direction += preconditioned_residual
        
        # 8. Speichern und Update
        iter_count += 1

        # Residuen-Norm für den Abbruch: ||r_k+1||_2
        residuals[iter_count] = norm(residual)
        
        gamma_old = gamma_new

    # Die resultierenden Vektoren sollen genau die richtige Laenge haben;
    # alle ueberschuessigen Elemente werden daher abgeschnitten:
    residuals = residuals[: iter_count + 1]

    return (solution, residuals)

### Teil 3: Vergleich der Verfahren

Nun wollen wir die implementierten Verfahren vergleichen. Dafür betrachten wir verschieden große Matrizen $A \in \mathbb{R}^{n \times n}$, wobei wir $n = 2^m + 1$ setzen. Für $m = 3$ ist $A$ also eine $9 \times 9$-Matrix, für $n=10$ hingegen schon $1025 \times 1025$. 

Testen Sie ihre Verfahren, in dem Sie die folgende Zelle ausführen:

In [ ]:
from util.plotting_06 import compute_and_plot_06

tol = 1.0e-9
m_smallest = 3
m_largest  = 12

n_iterations = np.zeros((m_largest - m_smallest, 2), dtype=int)
residuals = np.zeros((m_largest - m_smallest, 2))
norms = np.zeros((m_largest - m_smallest, 2))
times = np.zeros((m_largest - m_smallest, 3))

for i, m in enumerate(range(m_smallest, m_largest)):
    (
        cg_solution,
        cg_residuals,
        cg_norm,
        cg_time,
        pcg_solution,
        pcg_residuals,
        pcg_norm,
        pcg_time,
        init_time,
    ) = compute_and_plot_06(Av_mul, pre_mul, cg, pcg, m, tol, 2**m)
    if (type(cg_solution) == np.ndarray):
        n_iterations[i, 0] = cg_residuals.shape[0]
        residuals[i, 0] = cg_residuals[-1]
        norms[i, 0] = cg_norm
        times[i, 0]  = cg_time
    else:
        n_iterations[i, 0] = 0
        residuals[i, 0] = np.inf
        norms[i, 0] = np.inf
        times[i, 0]  = np.inf
    n_iterations[i, 1] = pcg_residuals.shape[0]
    residuals[i, 1] = pcg_residuals[-1]
    norms[i, 1] = pcg_norm
    times[i, 1]  = pcg_time
    times[i, 2]  = init_time

# Ausgabe der Tabellen
names = ["CG", "PCG"]
for i in range(2):
    if i == 1 and (type(cg_residuals) != np.ndarray):
        continue
    print(f"{names[i]}-Verfahren")
    print('m\titer\t||x-u||\t\t||b-Ax||\tct["]\t\tict["]')
    for j, m in enumerate(range(m_smallest, m_largest)):
        print(f"{m}\t{n_iterations[j,i]}\t{format(norms[j, i], '.2e')}\t{format(residuals[j, i], '.2e')}\t{format(times[j, i], '.1e')}\t\t{format(times[j, 2], '.1e')}")
    print("")